In [ ]:
# ✅ STEP 1: Imports
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications import EfficientNetB3
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D, Dropout
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint,ReduceLROnPlateau
from tensorflow.keras.applications import Xception
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D, Dropout
from tensorflow.keras.optimizers import Adam
import matplotlib.pyplot as plt
import os
import zipfile


In [ ]:


train_dir = '/kaggle/input/bdsl49-recognition-renamed-dataset/Recognition/train'
test_dir = '/kaggle/input/bdsl49-recognition-renamed-dataset/Recognition/test'



In [ ]:
import os

print(len(os.listdir(test_dir)))  # should print 48 if there are 48 class folders


In [ ]:

# ✅ STEP 2: Set Parameters
IMG_SIZE = (300, 300)
BATCH_SIZE = 32
NUM_CLASSES = 48


In [ ]:
# ✅ Training Data Generator
# ✅ Step 2: Augmentation & Data Generators
train_datagen = ImageDataGenerator(
    rescale=1./255,
    rotation_range=30,
    width_shift_range=0.2,
    height_shift_range=0.2,
    zoom_range=0.3,
    brightness_range=[0.8, 1.2],
    horizontal_flip=False,  # Hand signs are not symmetric
    fill_mode='nearest'
)


test_datagen = ImageDataGenerator(rescale=1./255)

train_generator = train_datagen.flow_from_directory(
    train_dir,
    target_size=IMG_SIZE,
    batch_size=BATCH_SIZE,
    class_mode='sparse'
)

test_generator = test_datagen.flow_from_directory(
    test_dir,
    target_size=IMG_SIZE,
    batch_size=BATCH_SIZE,
    class_mode='sparse'
)


In [ ]:
print(train_generator.class_indices)
print(test_generator.class_indices)


In [ ]:
# ✅ Load Pretrained Xception
base_model = Xception(weights='imagenet', include_top=False, input_shape=(300, 300, 3))
base_model.trainable = True  # Freeze base layers initially


In [ ]:
from tensorflow.keras.models import load_model


# Load the model
model = load_model("/kaggle/input/new18-keras/new20.keras")

In [ ]:
#model = Model(inputs=base_model.input, outputs=output)

In [ ]:
'''
for layer in base_model.layers[:50]:
    layer.trainable = False

# ✅ Step 4: Add Custom Classifier Head
x = base_model.output
x = GlobalAveragePooling2D()(x)
x = Dropout(0.5)(x)
x = Dense(256, activation='relu')(x)
x = Dropout(0.3)(x)
output = Dense(NUM_CLASSES, activation='softmax')(x)

model = Model(inputs=base_model.input, outputs=output)

# ✅ Step 5: Compile
model.compile(optimizer=Adam(learning_rate=1e-5),
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

model.summary()
'''

In [ ]:
# ✅ Step 6: Callbacks
checkpoint_cb = ModelCheckpoint('new20.keras', save_best_only=True, monitor='val_accuracy', mode='max', verbose=1)
earlystop_cb = EarlyStopping(monitor='val_accuracy', patience=7, restore_best_weights=True, verbose=1)
reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.5, patience=2, verbose=1, min_lr=1e-7)


In [ ]:
# ✅ Step 8: Train the Model
history = model.fit(
    train_generator,
    validation_data=test_generator,
    initial_epoch=30,
    epochs=50,
    callbacks=[checkpoint_cb, earlystop_cb, reduce_lr],
    
)

In [ ]:
import numpy as np
from sklearn.metrics import classification_report, confusion_matrix
import seaborn as sns


# Get true labels and predictions
test_generator.reset()
preds = model.predict(test_generator, steps=test_generator.samples // test_generator.batch_size + 1)
y_pred = np.argmax(preds, axis=1)  # If using softmax output
y_true = test_generator.classes[:len(y_pred)]  # Match lengths

# Optional: Class label mapping
label_map = {v: k for k, v in train_generator.class_indices.items()}  # reverse mapping
target_names = [label_map[i] for i in sorted(label_map.keys())]


# Print classification report
print(classification_report(y_true, y_pred, target_names=target_names))